In [ ]:
!pip install pdfminer.six
!pip install docx2txt
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=0e76499242af1f59b2789b27a7b3d9e9f4c098d2797fec2d9579d3aeb3c86061
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [ ]:
from pdfminer.high_level import extract_text


def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)


if __name__ == '__main__':
    print(extract_text_from_pdf('./Anya Carr Resume.pdf'))

FileNotFoundError: [Errno 2] No such file or directory: './Anya Carr Resume.pdf'

In [ ]:
import docx2txt


def extract_text_from_docx(docx_path):
    txt = docx2txt.process(docx_path)
    if txt:
        return txt.replace('\t', ' ')
    return None


if __name__ == '__main__':
    print(extract_text_from_docx('./Anya Carr Resume.docx'))

In [ ]:
!pip install requests

In [ ]:
import docx2txt
import nltk
import re
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')


def skill_exists(skill):
    url = f'https://api.apilayer.com/skills?q={skill}&count=1'
    headers = {'apikey': 'YOUR API KEY'}
    response = requests.request('GET', url, headers=headers)
    result = response.json()

    if response.status_code == 200:
        return len(result) > 0 and result[0].lower() == skill.lower()
    raise Exception(result.get('message'))

def extract_text_from_docx(docx_path):
    txt = docx2txt.process(docx_path)
    if txt:
        return txt.replace('\t', ' ')
    return None


def extract_names(txt):
    person_names = []

    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                person_names.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )

    return person_names

def extract_emails(resume_text):
    return re.findall(EMAIL_REG, resume_text)


def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)

    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))

    # we create a set to keep the results in.
    found_skills = set()

    # we search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)

    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in SKILLS_DB:
            found_skills.add(ngram)

    return found_skills

RESERVED_WORDS = [
    'school',
    'college',
    'univers',
    'academy',
    'faculty',
    'institute',
    'faculdades',
    'Schola',
    'schule',
    'lise',
    'lyceum',
    'lycee',
    'polytechnic',
    'kolej',
    'ünivers',
    'okul',
]

def extract_education(input_text):
  organizations = []

  # first get all the organization names using nltk
  for sent in nltk.sent_tokenize(input_text):
      for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
          if hasattr(chunk, 'label') and chunk.label() == 'ORGANIZATION':
              organizations.append(' '.join(c[0] for c in chunk.leaves()))

  # we search for each bigram and trigram for reserved words
  # (college, university etc...)
  education = set()
  for org in organizations:
      for word in RESERVED_WORDS:
          if org.lower().find(word) >= 0:
              education.add(org)

  return education



if __name__ == '__main__':
    text = extract_text_from_pdf('./Resume (2).pdf')
    #text = extract_text_from_docx('./TEST_RESUME.docx')
    names = extract_names(text)
    emails = extract_emails(text)
    skills = extract_skills(text)
    education_information = extract_education(text)

    if names:
        print(names[0])
        print(emails[0])
        print(skills)
        print(education_information)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'SKILLS_DB' is not defined

In [ ]:
import docx2txt
import nltk
import re
import requests
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')


def skill_exists(skill):
    #url = f'https://api.apilayer.com/skills?q={skill}' #&count=1
    url = f'https://api.apilayer.com/skills?q={skill}'
    headers = {'apikey': 'CG1GpriSzE0GL6UzYmXstckLZpvcryNY'}
    response = requests.request('GET', url, headers=headers)
    result = response.json()

    if response.status_code == 200:
        return len(result) > 0 and result[0].lower() == skill.lower()
    raise Exception(result.get('message'))



def extract_text_from_docx(docx_path):
    txt = docx2txt.process(docx_path)
    if txt:
        return txt.replace('\t', ' ')
    return None


def extract_names(txt):
    person_names = []

    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                person_names.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )

    return person_names

def extract_emails(resume_text):
    return re.findall(EMAIL_REG, resume_text)


def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)

    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))

    # we create a set to keep the results in.
    found_skills = set()

    # we search for each token in our skills database
    for token in filtered_tokens:
        if skill_exists(token.lower()):
            found_skills.add(token)

    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if skill_exists(ngram.lower()):
            found_skills.add(ngram)

    return found_skills


RESERVED_WORDS = [
    'school',
    'college',
    'univers',
    'academy',
    'faculty',
    'institute',
    'faculdades',
    'Schola',
    'schule',
    'lise',
    'lyceum',
    'lycee',
    'polytechnic',
    'kolej',
    'ünivers',
    'okul',
]

def extract_education(input_text):
  organizations = []

  # first get all the organization names using nltk
  for sent in nltk.sent_tokenize(input_text):
      for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
          if hasattr(chunk, 'label') and chunk.label() == 'ORGANIZATION':
              organizations.append(' '.join(c[0] for c in chunk.leaves()))

  # we search for each bigram and trigram for reserved words
  # (college, university etc...)
  education = set()
  for org in organizations:
      for word in RESERVED_WORDS:
          if org.lower().find(word) >= 0:
              education.add(org)

  return education



if __name__ == '__main__':
    text = extract_text_from_pdf('./Joshua Roasa Resume 2024.pdf')
    #text = extract_text_from_docx('./TEST_RESUME.docx')
    names = extract_names(text)
    emails = extract_emails(text)
    skills = extract_skills(text)
    education_information = extract_education(text)

    if names:
        print(names[0])
        print(emails[0])
        print(skills)
        print(education_information)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Joshua
roasa.joshua@gmail.com
{'Languages', 'Visual Studio Code', 'Computer Science', 'Database', 'Systems Programming', 'Human Computer Interaction', 'Design', 'Union', 'manager', 'Programming', 'Java', 'Networking', 'Excel', 'Studio', 'Algorithms', 'Management System', 'Fundamentals', 'Internet', 'clean', 'services', 'Donuts', 'environment', 'SOFT SKILLS', 'Analysis', 'Assembly', 'SKILLS', 'Linux', 'Visual Studio', 'Eclipse', 'Programming Languages', 'PowerPoint', 'HTML', 'University', 'Architecture', 'Teamwork', 'https', 'Microsoft', 'orders', 'Git', 'Time management', 'Management', 'C', 'Tagalog', 'Computer Architecture', 'framework', 'Object', 'management', 'English', 'Franchise', 'Operating', 'Principles', 'Windows', 'Communication', 'MySQL', 'Assembly Language', 'Operating System', 'Microsoft Word', 'Word', 'store', 'EDUCATION', 'Python', 'GPA', 'Code', 'Data', 'PHP', 'Manager', 'Software', 'workstation', 'General', 'Information Systems', 'Fluent', 'Inventory'}
{'Kean University